In [19]:
import torch
from torch import nn
import numpy
import pandas
from torch.utils.data import TensorDataset, DataLoader
from transformers import AutoConfig, AutoTokenizer, AutoModel

In [2]:
snapp_dataframe = pandas.read_csv(r"C:\Users\sadra\Desktop\Snappfood - Sentiment Analysis.csv",
                                  sep="\t",
                                  engine="python",
                                  on_bad_lines="skip")
digikala_dataframe = pandas.read_csv(r"C:\Users\sadra\Desktop\digikala_ds.csv",
                                  engine="python",
                                  on_bad_lines="skip")

In [3]:
snapp_dataframe.head(10)

Unnamed: 0                                            comment  label  \
0        NaN    واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح    SAD   
1        NaN  قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...  HAPPY   
2        NaN  قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...    SAD   
3        NaN  عالللی بود همه چه درست و به اندازه و کیفیت خوب...  HAPPY   
4        NaN                      شیرینی وانیلی فقط یک مدل بود.  HAPPY   
5        NaN             بدترین پیتزایی که تا به حال خورده بودم    SAD   
6        NaN                            از همه لحاظ عالی ممنونم  HAPPY   
7        NaN  کیفیت غذا متوسط رو به پایین بود انگار داخل یه ...    SAD   
8        NaN  همه اقلام تازه و به روز وخیلیییییی سریع بدستم ...  HAPPY   
9        NaN  همه چی خوب ولی هات داگ دورش کلا سوخته بود و دا...    SAD   

   label_id  
0       1.0  
1       0.0  
2       1.0  
3       0.0  
4       0.0  
5       1.0  
6       0.0  
7       1.0  
8       0.0  
9       1.0

In [4]:
snapp_dataframe = snapp_dataframe.drop(columns=["Unnamed: 0"])

In [5]:
digikala_dataframe.head(10)

Text  Score  Suggestion
0  این اولین تجربه من برای خرید ایفون هست   امروز...    100           1
1                     خرید این محصول رو توصیه میکنم      84           1
2  1 ساله این گوشی رو دارم   هیچ نقطه ضعفی ازش ند...     60           1
3  سلام خدمت دوستان این گوشی از همه نظر عالی  کیف...     96           1
4  سلام دوستانی که نگران شکستن صفحه نمایش هستند ا...     92           1
5  من پکیج کاملش روخریدم   ازهرنظرعالیه   هیچ نقد...     60           1
6  اول از همه خوشحالم که بالاخره نسخه های 128 گیگ...     96           1
7  من دو ماهه خریدم واقعا راضیم  دوربینشم عالیه  ...    100           1
8  بنظر من گوشی خوب و مقرون به صرفه ای بود توی ای...    100           1
9  از خریدم راضی ام و گوشی بینظیریه  منتهی یه ایر...     92           1

In [6]:
print(f"snapp dataset length: {snapp_dataframe.size}")
print(f"digikala dataset length: {digikala_dataframe.size}")

snapp dataset length: 209994
digikala dataset length: 9783


In [7]:
print(f"snapp:{snapp_dataframe.info()}")
print(f"digikala:{digikala_dataframe.info()}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69998 entries, 0 to 69997
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   comment   69998 non-null  object 
 1   label     69998 non-null  object 
 2   label_id  69481 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1.6+ MB
snapp:None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3261 entries, 0 to 3260
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Text        3261 non-null   object
 1   Score       3261 non-null   int64 
 2   Suggestion  3261 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 76.6+ KB
digikala:None


In [8]:
digikala_dataframe["Suggestion"].value_counts()

Suggestion
1    2382
3     460
2     419
Name: count, dtype: int64

in the last kernel, i've seen that dataset is clearly small (even thought its okay for sentiment classification)
and also it has big amount of gap for labels, we can fix that but because its a small dataset it doesnt worth the time
so we will leave the digikala dataset there

In [9]:
def helper(x):
    x = 1 if x == "HAPPY" else 0
    return x
snapp_dataframe["label"] = snapp_dataframe["label"].apply(helper)

In [10]:
print(snapp_dataframe["label"])

0        0
1        1
2        0
3        1
4        1
        ..
69993    0
69994    0
69995    1
69996    0
69997    1
Name: label, Length: 69998, dtype: int64


In [11]:
snapp_y = snapp_dataframe["label"].to_list()

In [12]:
snapp_X = snapp_dataframe["comment"].to_list()

In [13]:
print(f"snapp y lenght: {len(snapp_y)}")
print(f"snapp X lenght: {len(snapp_X)}")

snapp y lenght: 69998
snapp X lenght: 69998


In [14]:
keyword = "HooshvareLab/bert-fa-base-uncased"
config = AutoConfig.from_pretrained(keyword)
persian_bert = AutoModel.from_pretrained(keyword, config=config)
persian_bert_tokenizer = AutoTokenizer.from_pretrained(keyword)

#persian bert is the pretrained embedding model on large persian dataset
#hugging face: https://huggingface.co/HooshvareLab/bert-fa-base-uncased
#github: https://github.com/hooshvare/parsbert

In [16]:
class sentimentClassifier(nn.Module):
    def __init__(self, input_dim: int,
                 hidden_dim1: int, hidden_dim2: int,
                 output_dim:int, embedder, tokenizer):
        
        super(sentimentClassifier, self).__init__()
        
        from torch.utils.data import TensorDataset, DataLoader
        self.layer1 = nn.Linear(input_dim, hidden_dim1)
        self.act1 = nn.ReLU()
        self.layer2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(hidden_dim2, output_dim)
        self.embedder = embedder
        self.tokenizer = tokenizer
        self.dataset = TensorDataset
        self.dataloader = DataLoader
        
    def forward(self, x):
        """
        inputs: 
        
        x, the embedded torch tensor for sentences

        output:

        prediction logits
        """
        output_1_withoutReLU = self.layer1(x)
        output_1 = self.act1(output_1_withoutReLU)
        output_2_withoutReLU = self.layer2(output_1)
        output_2 = self.act2(output_2_withoutReLU)
        prediction = self.layer3(output_2)
        return prediction        
    def tokenize(self,
                x: list,device: str="cpu",
                sentence_maxLength: int=50,
                batch_size: int=10):
        """
        inputs: 
        x, a list of sentences
        
        device, the device to do the tokenization task in it
        
        sentence_maxLength, length of every sentence to padd, truncate to
        
        batch_size, length of every batcg in tokenization

        output:
        
        DataLoader class of torch with structure of (input_ids, attention_mask)
        """
        
        self.embedder.to(device)
        self.embedder.eval()

        tokenized = self.tokenizer(x,
                            padding="max_length",
                            max_length=sentence_maxLength,
                            truncation=True,
                            return_tensors="pt")
        
        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]

        # Create a DataLoader to automatically handle batching
        dataset = self.dataset(input_ids, attention_mask)
        dataloader = self.dataloader(dataset, batch_size=batch_size)

        return dataloader
    def embedd(self, dataloader, device: str="cpu"):
        """
        inputs: 
        
        dataloader, pytorch DataLoader class with input_ids, attention_mask
        device, device to use for embedding task, (cpu or cuda)

        output: 
        
        torch tensor in shape of (dataloader_length, 768 (embedding size))
        """
        from tqdm import tqdm as progressbar
        embeddings_list = []

        with torch.inference_mode():
            for input_ids_batch, attention_mask_batch in progressbar(dataloader, desc="Embedding batches"):
                input_ids_batch = input_ids_batch.to(device)
                attention_mask_batch = attention_mask_batch.to(device)
                
                outputs = self.embedder(input_ids=input_ids_batch, attention_mask=attention_mask_batch)
                cls_embeddings = outputs.pooler_output.cpu()
                embeddings_list.append(cls_embeddings)

        ready_x = torch.cat(embeddings_list, dim=0)
        return ready_x
    def predict(self,x: str,
                device: str="cpu",
                return_soft_fa_prediction: bool=False,
                return_soft_en_prediction: bool=False):
        if return_soft_en_prediction and return_soft_fa_prediction:
            raise ValueError("Only one of 'return_soft_fa_prediction' or 'return_soft_en_prediction' can be True.")
        """
        input: 
        
        x, as a single string representing the sentence
        device, the device to do the full prediction task in it
        return_soft_fa_prediction, decides that function returns persian soft predciton or not
        return_soft_en_prediction, decides that function returns english soft prediction or not
        
        output:

        a single integer representing the classified mode, {0: "negative", 1: "positive"} or,
        a str representing 'positive' or 'negative' in english or,
        a str representing 'positive' or 'negative' in persian
        """
        self.to(device)
        self.eval()
        decode_list = ["negative", "positive"]
        with torch.inference_mode():
            tokenized = self.tokenize([x], device=device)
            embedded = self.embedd(tokenized, device=device)
            pred = self.forward(embedded)
            prediction_rounded = round(torch.sigmoid(pred).squeeze().item())
        if return_soft_en_prediction:
            return decode_list[prediction_rounded] 
        elif return_soft_fa_prediction:
            if prediction_rounded == 0:
                return "منفی"
            return "مثبت"
        else:
            return prediction_rounded

In [17]:
model = sentimentClassifier(768, 128, 64, 1, persian_bert, persian_bert_tokenizer)

In [48]:
def fit(model, dataloader,
        loss_function, optimizer,
        epochs: int, savepath: str,
        device="cpu"):
    
    from tqdm import tqdm as progressbar
    model.to(device)
    model.train()
    
    for epoch in range(epochs):
        loss_sum = 0
        for embedding_batch, label_batch in progressbar(dataloader, desc=f"Model training epoch: {epoch}"):
            prediction = model(embedding_batch).to(torch.float32)
            if prediction.shape != label_batch.shape:
                label_batch = label_batch.view_as(prediction)
            loss = loss_function(prediction.to(torch.float32), label_batch.to(torch.float32))
            loss_sum += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        avg_loss = loss_sum / len(dataloader)
        progressbar.write(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")
        torch.save(model.state_dict(), savepath) # saving models parameter dictionary (like a callback)
        progressbar.write(f"model state dict saved to: {savepath}")
        progressbar.write("-----------------------------------------------------------------")

training

In [21]:
crite = nn.BCEWithLogitsLoss() # loss function 
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) # models optimizer

In [22]:
tokenized = model.tokenize(snapp_X) # tokenized sentences, as DataLoader
print("tokenization done")
embedded = model.embedd(tokenized) # embedded tensors
print("embedding done")

tokenization done


Embedding batches: 100%|█████████████████████████████████████████████████████████| 7000/7000 [1:39:56<00:00,  1.17it/s]

embedding done


In [24]:
X_dataset =  TensorDataset(embedded, torch.tensor(snapp_y)) # creating dataset instance
X_dataloader = DataLoader(X_dataset)# creating DataLoader instance for fit function

In [49]:
fit(model, X_dataloader, crite,
    optimizer, 10, 
    r"C:\Users\sadra\SadraProjects\telegram_bot_project\telegram_sentiment_model_weights.pth") # running training loop

Model training epoch: 0: 100%|██████████████████████████████████████████████████| 69998/69998 [03:08<00:00, 371.86it/s]


Epoch 1, Loss: 0.3929
model state dict saved to: C:\Users\sadra\SadraProjects\telegram_bot_project\telegram_sentiment_model_weights.pth
-----------------------------------------------------------------


Model training epoch: 1: 100%|██████████████████████████████████████████████████| 69998/69998 [03:08<00:00, 370.97it/s]


Epoch 2, Loss: 0.3882
model state dict saved to: C:\Users\sadra\SadraProjects\telegram_bot_project\telegram_sentiment_model_weights.pth
-----------------------------------------------------------------


Model training epoch: 2: 100%|██████████████████████████████████████████████████| 69998/69998 [03:11<00:00, 364.82it/s]


...

Epoch 8, Loss: 0.3688
model state dict saved to: C:\Users\sadra\SadraProjects\telegram_bot_project\telegram_sentiment_model_weights.pth
-----------------------------------------------------------------


Model training epoch: 8: 100%|██████████████████████████████████████████████████| 69998/69998 [03:09<00:00, 369.78it/s]


Epoch 9, Loss: 0.3661
model state dict saved to: C:\Users\sadra\SadraProjects\telegram_bot_project\telegram_sentiment_model_weights.pth
-----------------------------------------------------------------


Model training epoch: 9: 100%|██████████████████████████████████████████████████| 69998/69998 [03:08<00:00, 371.42it/s]


Epoch 10, Loss: 0.3639
model state dict saved to: C:\Users\sadra\SadraProjects\telegram_bot_project\telegram_sentiment_model_weights.pth
-----------------------------------------------------------------


In [55]:
torch.save(model, r"C:\Users\sadra\SadraProjects\telegram_bot_project\telegram_sentiment_full_model.pth")